In [47]:
import pandas as pd
import numpy as np
import Levenshtein as lev
import random
import re
#import csv
from functools import reduce
import itertools
from scipy.stats import skew
import missingno as msno
import string
#from datetime import datetime as dt

In [48]:
import warnings
warnings.filterwarnings("ignore")

In [49]:
pd.set_option('display.max_rows', 5000)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 1000)

In [50]:
incidents = pd.read_csv("data/gun-violence-data_01-2013_03-2018.csv")

In [51]:
lv_incident = pd.read_csv("data/las-vegas-1-10-2017.csv")
incidents = pd.concat([incidents, lv_incident])

In [52]:
incidents.head(3)

,incident_id,date,state,city_or_county,address,n_killed,n_injured,incident_url,source_url,incident_url_fields_missing,congressional_district,gun_stolen,gun_type,incident_characteristics,latitude,location_description,longitude,n_guns_involved,notes,participant_age,participant_age_group,participant_gender,participant_name,participant_relationship,participant_status,participant_type,sources,state_house_district,state_senate_district
0,461105,2013-01-01,Pennsylvania,Mckeesport,1506 Versailles Avenue and Coursin Street,0,4,http://www.gunviolencearchive.org/incident/461105,http://www.post-gazette.com/local/south/2013/0...,False,14.0,NaN,NaN,Shot - Wounded/Injured||Mass Shooting (4+ vict...,40.3467,NaN,-79.8559,NaN,Julian Sims under investigation: Four Shot and...,0::20,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male||1::Male||3::Male||4::Female,0::Julian Sims,NaN,0::Arrested||1::Injured||2::Injured||3::Injure...,0::Victim||1::Victim||2::Victim||3::Victim||4:...,http://pittsburgh.cbslocal.com/2013/01/01/4-pe...,NaN,NaN
1,460726,2013-01-01,California,Hawthorne,13500 block of Cerise Avenue,1,3,http://www.gunviolencearchive.org/incident/460726,http://www.dailybulletin.com/article/zz/201301...,False,43.0,NaN,NaN,"Shot - Wounded/Injured||Shot - Dead (murder, a...",33.9090,NaN,-118.3330,NaN,Four Shot; One Killed; Unidentified shooter in...,0::20,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male,0::Bernard Gillis,NaN,0::Killed||1::Injured||2::Injured||3::Injured,0::Victim||1::Victim||2::Victim||3::Victim||4:...,http://losangeles.cbslocal.com/2013/01/01/man-...,62.0,35.0
2,478855,2013-01-01,Ohio,Lorain,1776 East 28th Street,1,3,http://www.gunviolencearchive.org/incident/478855,http://chronicle.northcoastnow.com/2013/02/14/...,False,9.0,0::Unknown||1::Unknown,0::Unknown||1::Unknown,"Shot - Wounded/Injured||Shot - Dead (murder, a...",41.4455,Cotton Club,-82.1377,2.0,NaN,0::25||1::31||2::33||3::34||4::33,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male||1::Male||2::Male||3::Male||4::Male,0::Damien Bell||1::Desmen Noble||2::Herman Sea...,NaN,"0::Injured, Unharmed, Arrested||1::Unharmed, A...",0::Subject-Suspect||1::Subject-Suspect||2::Vic...,http://www.morningjournal.com/general-news/201...,56.0,13.0


In [53]:
incidents = incidents.drop(["incident_url_fields_missing", "congressional_district", "location_description", "notes", "incident_url", "source_url", "sources"], axis=1)

In [54]:
incidents["date"] = pd.to_datetime(incidents["date"])
incidents = incidents.sort_values(by=["date"])
# Remove incidents from 2013
incidents = incidents[incidents["date"].dt.year != 2013].reset_index().drop("index", axis=1)

Each individual incident before March 2018 contains information about all participants in it: **participant_name**, **participant_age**, **participant_age_group**, **participant_gender**, **participant_relationship**, **participant_status**, and **participant_type**. In the event that multiple participants took part in one incident, the raw data comprises the features of multiple participants in a string dictionary format: \\

\begin{gather}
{participant_1}:{value_1} || \ {participant_2}:{value_2} || \ \cdots
\end{gather}

Likewise, features of the guns involved in the incident such as **gun_stolen** and **gun_type** may also accont for multiple guns used in an incident by using the same type of formatting: \\

\begin{gather}
{gun_1}:{value_1} || \ {gun_2}:{value_2} || \ \cdots
\end{gather}

Lastly, the **incident_characteristic** column has a similar format, the only difference being that it requires a list string instead of a dictionary: \\

\begin{gather}
{characteristic_1} || \ {characteristic_2} || \ \cdots
\end{gather}

The next section converts these formats to the standard Python dictionary or list format.

In [55]:
def split_into_dict(string):
    if pd.isnull(string):
        return np.nan
    str_dict = dict()
    # The dataset contains instances separated by | or ||, and : or :: respectively
    # Thus we have to consider both cases when working on those strings
    str_list = '§'.join('§'.join(string.split('||')).split('|')).split('§')
    for index in str_list:
        # If indices and values are split by '::'
        i, sep, v = index.partition('::')
        if sep and v:
            str_dict[int(i)] = v
        # If indices and values are split by ':'
        else:
            i, _, v = index.partition(':')
            str_dict[int(i)] = v
    return str_dict

In [56]:
def convert_vals_to_int(str_dict):
    if pd.isnull(str_dict):
        return np.nan
    for key in str_dict:
        str_dict[key] = int(str_dict[key])
    return str_dict

In [57]:
def split_into_list(string):
    if pd.isnull(string):
        return np.nan
    # Split the string by '|' or '||', depending on the format
    return '#$%'.join('#$%'.join(string.split('||')).split('|')).split('#$%')

In [58]:
for col in ['gun_stolen', 'gun_type', 'participant_name', 'participant_age', 'participant_age_group', 'participant_gender', 'participant_relationship', 'participant_status', 'participant_type']:
    incidents[col] = incidents[col].apply(split_into_dict)
incidents['participant_age'] = incidents['participant_age'].apply(convert_vals_to_int)
incidents['incident_characteristics'] = incidents['incident_characteristics'].apply(split_into_list)

The next task is to split the raw dataset in 3: one containing all **incidents** with incident characteristics as features, i.e. date, state, city/county etc., one containing all **guns** involved in the incidents, with type and origin as features, and lastly a dataset with all **participants** and their details (age, status etc.)

In [59]:
df_guns = incidents[["gun_type", "gun_stolen", "incident_id"]].copy()

In [60]:
# Create a NumPy array to store gun information, then converting it to DataFrame for performance improvement
guns_array = np.empty(shape=(int(incidents["n_guns_involved"].sum()), len(df_guns.columns)), dtype=object)
rows = df_guns.values
array_i = 0
for df_i in range(df_guns.shape[0]):
    if pd.notnull(rows[df_i][df_guns.columns.get_loc("gun_type")]):
        for k, v in rows[df_i][df_guns.columns.get_loc("gun_type")].items():
            if pd.notnull(rows[df_i][df_guns.columns.get_loc("gun_stolen")]):
                if k in rows[df_i][df_guns.columns.get_loc("gun_stolen")].keys():
                    guns_array[array_i][df_guns.columns.get_loc("gun_stolen")] = rows[df_i][df_guns.columns.get_loc("gun_stolen")][k]
            guns_array[array_i][df_guns.columns.get_loc("gun_type")] = v
            guns_array[array_i][df_guns.columns.get_loc("incident_id")] = rows[df_i][df_guns.columns.get_loc("incident_id")]
            array_i += 1
guns = pd.DataFrame(guns_array, columns=["type", "stolen", "incident_id"])

In [61]:
df_participants = incidents[["participant_name", "participant_age", "participant_age_group", "participant_gender", "participant_relationship", "participant_status", "participant_type", "incident_id"]].copy()

In [62]:
n_participants = 0
for row in incidents.values:
    if pd.notnull(row[incidents.columns.get_loc("participant_type")]):
        n_participants += len(row[incidents.columns.get_loc("participant_type")])

In [63]:
participants_array = np.empty(shape=(n_participants, len(df_participants.columns)), dtype=object)
rows = df_participants.values
array_i = 0
for df_i in range(df_participants.shape[0]):
    if pd.notnull(rows[df_i][df_participants.columns.get_loc("participant_type")]):
        for k, v in rows[df_i][df_participants.columns.get_loc("participant_type")].items():
            for col in ["participant_name", "participant_age"]:
                if pd.notnull(rows[df_i][df_participants.columns.get_loc(col)]):
                    if k in rows[df_i][df_participants.columns.get_loc(col)].keys():
                        participants_array[array_i][df_participants.columns.get_loc(col)] = rows[df_i][df_participants.columns.get_loc(col)][k]
            if pd.notnull(rows[df_i][df_participants.columns.get_loc("participant_age_group")]):
                if k in rows[df_i][df_participants.columns.get_loc("participant_age_group")].keys():
                    # Some age_group values of the same value have different formats, i.e. "Teen (12-17)" and "Teen 12-17"
                    if rows[df_i][df_participants.columns.get_loc("participant_age_group")][k] == "Teen (12-17)":
                        participants_array[array_i][df_participants.columns.get_loc("participant_age_group")] = "Teen 12-17"
                    else:
                        participants_array[array_i][df_participants.columns.get_loc("participant_age_group")] = rows[df_i][df_participants.columns.get_loc("participant_age_group")][k]
            for col in ["participant_gender", "participant_relationship", "participant_status"]:
                if pd.notnull(rows[df_i][df_participants.columns.get_loc(col)]):
                    if k in rows[df_i][df_participants.columns.get_loc(col)].keys():
                        participants_array[array_i][df_participants.columns.get_loc(col)] = rows[df_i][df_participants.columns.get_loc(col)][k]
            participants_array[array_i][df_participants.columns.get_loc("participant_type")] = v
            participants_array[array_i][df_participants.columns.get_loc("incident_id")] = rows[df_i][df_participants.columns.get_loc("incident_id")]
            array_i += 1     
participants = pd.DataFrame(participants_array, columns=["name", "age", "age_group", "gender", "relationship", "status", "type", "incident_id"])

In [64]:
incidents = incidents[["incident_id", "date", "state", "city_or_county", "address", "latitude", "longitude", "n_killed", "n_injured", "n_guns_involved", "incident_characteristics"]]

In [65]:
i_characs = []
for row in incidents.values:
    if isinstance(row[incidents.columns.get_loc("incident_characteristics")], list):
        for charac in row[incidents.columns.get_loc("incident_characteristics")]:
            if charac not in i_characs:
                i_characs.append(charac)

In [66]:
characteristics = ["non_shooting", "mass_shooting", "school_shooting", "accidental", "defensive", "child_involved", "illegal_possession", "suicide"]

In [67]:
for c in characteristics:
    incidents[c] = 0

In [68]:
index = 0
for row in incidents.values:
    if isinstance(row[incidents.columns.get_loc("incident_characteristics")], list):
        for charac in row[incidents.columns.get_loc("incident_characteristics")]:
            first_word = charac.split(" ")[0]
            if charac == "Non-Shooting Incident":
                incidents.at[index, "non_shooting"] = 1
            elif charac == "Gun(s) stolen from owner" or charac == "Unlawful purchase/sale" or charac == "Criminal act with stolen gun" or charac == "Under the influence of alcohol or drugs (only applies to the subject/suspect/perpetrator )" or charac == "Possession of gun by felon or prohibited person" or charac == "Stolen/illegally owned gun{s} recovered during arrest/warrant" or charac == "Guns stolen from law enforcement":
                incidents.at[index, "illegal_possession"] = 1
            elif charac == "Suicide^" or charac == "Suicide" or charac == "Murder/Suicide":
                incidents.at[index, "suicide"] = 1
            elif first_word == "Accidental" or first_word == "Accidental/Negligent":
                incidents.at[index, "accidental"] = 1
            elif first_word == "Mass":
                incidents.at[index, "mass_shooting"] = 1
            elif first_word == "School":
                incidents.at[index, "school_shooting"] = 1
            elif first_word == "Defensive":
                incidents.at[index, "defensive"] = 1
            elif first_word == "Child":
                incidents.at[index, "child_involved"] = 1
    index += 1

Addresses in the dataset contain abbreviations that are not applied universally, meaning that some entries might not abbreviate common words, e.g. Street, while other might do it in a different way, e.g. St, Str, or using punctuation: St., Str., etc. The following cells are looking to identify such abbreviations by computing the Levenshtein distance between addresses and observing identical addressed written in different formats.

In [69]:
def tidy_whitespaces(address):
    new_address = re.sub(' +', ' ', address)
    new_address = re.sub(' ,', ',', new_address)
    new_address = re.sub(r'^\s+', '', new_address, flags=re.UNICODE)
    new_address = re.sub(r"\s+$", "", new_address, flags=re.UNICODE)
    return new_address

In [70]:
def remove_special_characters(address):
    new_address = re.sub("\.", "", address)
    new_address = re.sub(",|\-|\/|#", " ", new_address)
    return new_address

In [71]:
def clean_address_string(address):
    if pd.isnull(address):
        return np.nan
    else:
        new_address = remove_special_characters(address)
        # Apply lookup dictionary rules
        new_address = reduce(lambda x, y: re.sub(y, lookup_dict[y], x), lookup_dict, new_address)
        new_address = tidy_whitespaces(new_address)
        return new_address

In [72]:
def strip_address(address):
    address = re.sub('([0-9]*th|[0-9]*1st|[0-9]*2nd|[0-9]*3rd)', ' ', address)
    address = re.sub('[0-9]*', '', address)
    address = re.sub('N |E |S |W |NE|NW|SE|SW', ' ', address)
    address = tidy_whitespaces(address)
    return address

In [73]:
lookup_dict = {"Pkwy": "Parkway", "Blvd": " Boulevard", "Rd": "Road", "Dr ": "Drive ", "Dr$": "Drive", "Ave ": "Avenue ", "Ave$": "Avenue", "avenues": "Avenues", "Block": "block", "Expwy": "Expressway", "Pl ": "Place ", "Pl$": "Place", "Ln": "Lane", "Northwest": "NW", "Southwest": "SW", "Northeast": "NE", "Southeast": "SE", "North-west": "NW", "South-west": "SW", "North-east": "NE", "South-east": "SE", "NorthWest": "NW", "SouthWest": "SW", "NorthEast": "NE", "SouthEast": "SE", "North-West": "NW", "South-West": "SW", "North-East": "NE", "South-East": "SE", "North ": "N ", "South ": "S ", "East ": "E ", "West ": "W ", "North$": "N", "South$": "S", "East$": "E", "West$": "W", "Av ": "Avenue ", "Av$": "Avenue", "St$": "Street"}

In [74]:
lev_incidents = incidents.sample(frac=0.01)

In [75]:
index = 0
lev_addresses = dict()
for item in list(set(lev_incidents[pd.notnull(lev_incidents["address"])]["address"].apply(clean_address_string))):
    lev_addresses[index] = strip_address(item)
    index += 1

In [76]:
# Find combinations of addresses that have between 0.95 and 0.99 similarity, disregarding numbers of streets and cardinals
for comb in itertools.combinations(lev_addresses, 2):
    similarity = lev.ratio(lev_addresses[comb[0]], lev_addresses[comb[1]])
    if similarity*100 in range(95, 100):
            print("There is a similarity of " + str(similarity) + " between the following strings:")
            print(" 1. " + lev_addresses[comb[0]])
            print(" 2. " + lev_addresses[comb[1]])

There is a similarity of 0.96 between the following strings:
 1. block of Martin Lu er King Jr Boulevard
 2. block of Martin Lu er King Boulevard
There is a similarity of 0.96 between the following strings:
 1. Julian Street
 2. Julia Street
There is a similarity of 0.95 between the following strings:
 1. block of Linn Street
 2. block of Lynn Street
There is a similarity of 0.95 between the following strings:
 1. block of Cleo Avenue
 2. block of Clio Avenue
There is a similarity of 0.96 between the following strings:
 1. Greene Street
 2. Green Street


In [77]:
incidents['address'] = incidents['address'].apply(clean_address_string)

In [78]:
def fix_county_format(string):
    if pd.notnull(string):
        return string.replace("County", "(county)")

In [79]:
incidents["city_or_county"] = incidents["city_or_county"].apply(fix_county_format)

In [80]:
incidents.at[incidents["city_or_county"] == "jefferson parish (county)", "city_or_county"] = "Jefferson Parish (county)"

In [81]:
incidents = incidents.reset_index()
del incidents["index"]
del incidents["incident_characteristics"]

In [82]:
participants.at[participants["age"] == 311, "age"] = 31
participants.at[participants["age"] == 209, "age"] = 33

In [83]:
for sts in ["Injured, Unharmed", "Injured, Unharmed, Arrested", "Killed, Arrested", "Killed, Injured", "Killed, Unharmed", "Killed, Unharmed, Arrested"]:
    participants.at[participants["status"] == sts, "status"] = None

In [84]:
participants.at[participants["gender"] == "Male, female", "gender"] = "Female"

In [85]:
# name is an address
for index in participants[participants["name"] == "13300 Block of Frederick St"].index:
    incidents.at[incidents["incident_id"] == participants.loc[index]["incident_id"], "address"] = "13300 Block of Frederick Steet"
# name is a number, age missing
for number in [20, 26, 27]:
    for index in participants[participants["name"] == str(number)].index:
        participants.at[index, "age"] = number
for number in range(31, 54):
    for index in participants[participants["name"] == str(number)].index:
        participants.at[index, "age"] = number
# name contains other type of information
for index in participants[participants["name"] == "brother"].index:
    participants.at[index, "relationship"] = "Family"
for index in participants[participants["name"] == "cousin"].index:
    participants.at[index, "relationship"] = "Family"
for index in participants[participants["name"] == "son-in-law"].index:
    participants.at[index, "relationship"] = "Family"
for index in participants[participants["name"] == "coach"].index:
    participants.at[index, "name"] = "Aquaintance"
for index in participants[participants["name"] == "ex-husband"].index:   
    participants.at[index, "relationship"] = "Significant others - current or former"
for index in participants[participants["name"] == "girlfriend"].index:   
    participants.at[index, "relationship"] = "Significant others - current or former"
for index in participants[participants["name"] == "his wife"].index:
    participants.at[index, "relationship"] = "Significant others - current or former"
# remove the dogs from the dataset
participants = participants.drop(participants[participants["name"] == "dog"].index)
participants = participants.drop(participants[participants["name"] == "pit bull"].index)

In [86]:
participants = participants.reset_index()
del participants["index"]

In [87]:
guns = guns.replace({"Unknown": None})

In [88]:
guns["category"] = guns["type"].map({"10mm": "Pistol", "12 gauge": "Shotgun", "16 gauge": "Shotgun", "20 gauge": "Shotgun", "22 LR": "Rifle", "223 Rem [AR-15]": "Rifle", "25 Auto": "Pistol", "28 gauge": "Shotgun", "30-06 Spr": "Rifle", "30-30 Win": "Rifle", "300 Win": "Rifle", "308 Win": "Rifle", "32 Auto": "Pistol", "357 Mag": "Handgun", "38 Spl": "Handgun", "380 Auto": "Pistol", "40 SW": "Pistol", "410 gauge": "Shotgun", "44 Mag": "Handgun", "45 Auto": "Pistol", "7.62 [AK-47]": "Assault Rifle", "9mm": "Pistol", "Handgun": "Handgun", "Other": None, "Rifle": "Rifle", "Shotgun": "Shotgun"})

In [89]:
incidents.to_csv("incidents_vis.csv")
participants.to_csv("participants_vis.csv")
guns.to_csv("guns_vis.csv")

In [90]:
del incidents["address"]
del incidents["latitude"]
del incidents["longitude"]
del incidents["n_guns_involved"]

In [91]:
del participants["name"]